Optimisation 1:
Question a)

In [59]:
import pennylane as qml
from jax import numpy as np
import jax

def ground(params):
    X = np.array([[0, 1], [1, 0]])
    Y = np.array([[0, -1j], [1j, 0]])
    Z = np.array([[1, 0], [0, -1]])
    H = params[0]*X + params[1]*Y + params[2]*Z
    eigenvalues, eigenvectors = np.linalg.eigh(H)
    groundenergy = eigenvalues[0]
    groundvector = eigenvectors[:, 0]
    return groundvector

params = np.array([0.5, 1.0, 1.0])
ground(params)

Array([-0.40824828+0.j        ,  0.4082482 +0.81649655j], dtype=complex64)

Ground state found from direct diagonlisation of Hamiltonian with parameters [0.5, 1.0, 1.0] is [-0.40824828+0.j, 0.4082482 +0.81649655j].

Question b)

In [60]:
dev1 = qml.device("lightning.qubit", wires=1)
optimise = np.array([0.1, 0.1])
ops = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

@qml.qnode(dev1)
def circuit(optimise):
    qml.RY(optimise[0], wires=0)
    qml.RZ(optimise[1], wires=0)
    return qml.expval(qml.Hamiltonian(params, ops))

In [61]:
dcircuit = jax.grad(circuit, argnums=0)
print(dcircuit(optimise))

[0.4945179  0.09435131]


In [62]:
def cost(x):
    return circuit(x)

In [63]:
print(cost(optimise))

1.0546383


In [64]:
import jaxopt

# initialise the optimizer
opt = jaxopt.GradientDescent(cost, stepsize=0.4, acceleration = False)

# set the number of steps
steps = 200
# set the initial parameter values
opt_state = opt.init_state(optimise)

for i in range(steps):
    # update the circuit parameters
    optimise, opt_state = opt.update(optimise, opt_state)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(optimise)))

print("Optimized parameters: {}".format(optimise))
print("Ground State from Diagonalisation: " + str(ground(params)))
print("Ground State from Optimisation: " + str(np.array([np.cos(optimise[0]/2), np.exp(1j*optimise[1])*np.sin(optimise[0]/2)])))

/usr/local/lib/python3.10/dist-packages/pennylane/devices/preprocess.py:284: UserWarning: Differentiating with respect to the input parameters of LinearCombination is not supported with the adjoint differentiation method. Gradients are computed only with regards to the trainable parameters of the circuit.

 Mark the parameters of the measured observables as non-trainable to silence this warning.
  warnings.warn(


Cost after step     5: -1.1575691
Cost after step    10: -1.4993813
Cost after step    15: -1.4999897
Cost after step    20: -1.4999999
Cost after step    25: -1.5000000
Cost after step    30: -1.5000000
Cost after step    35: -1.5000000
Cost after step    40: -1.5000000
Cost after step    45: -1.5000000
Cost after step    50: -1.5000000
Cost after step    55: -1.5000000
Cost after step    60: -1.5000000
Cost after step    65: -1.5000000
Cost after step    70: -1.5000000
Cost after step    75: -1.5000000
Cost after step    80: -1.5000000
Cost after step    85: -1.5000000
Cost after step    90: -1.5000000
Cost after step    95: -1.5000000
Cost after step   100: -1.5000000
Cost after step   105: -1.5000000
Cost after step   110: -1.5000000
Cost after step   115: -1.5000000
Cost after step   120: -1.5000000
Cost after step   125: -1.5000000
Cost after step   130: -1.5000000
Cost after step   135: -1.5000000
Cost after step   140: -1.5000000
Cost after step   145: -1.5000000
Cost after ste

Question c)

Ground state found from optimisation is the same as found from diagonlisation, with a global phase difference, and both give the same ground energy. For a simple Hamiltonian such as this problem, direct diagonalisation of the Hamiltonian was a much faster and simpler method to find the ground state. As the Hamiltonian increases in size, however, the diagonlisation would become a very difficult task, and the optimisation method would find the ground state more efficiently. This is due to the Hamiltonian of the system increasing exponentially with the amount of qubits; a classical algorithm will not be able to handle this past 50 qubits, and would be very slow up to that point.